In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
pd.options.mode.chained_assignment = None
import glob

In [2]:
def read_file_to_df(path):
    all_files = glob.glob(path + '/*.csv')
    df = pd.DataFrame()
    for filename in all_files:
        temp = pd.read_csv(filename)
        df = pd.concat([df, temp])
    return df



In [3]:
bokhoven = engelerschans = read_file_to_df('C:/Users/20175848/Documents/waterschap-datachallenge/data/den bosch/sewer_data_db/flow/Bokhoven/')
de_rompert = read_file_to_df('C:/Users/20175848/Documents/waterschap-datachallenge/data/den bosch/sewer_data_db/flow/de Rompert/')
haarsteeg = read_file_to_df('C:/Users/20175848/Documents/waterschap-datachallenge/data/den bosch/sewer_data_db/flow/Haarsteeg/')
helftheuvelweg = read_file_to_df('C:/Users/20175848/Documents/waterschap-datachallenge/data/den bosch/sewer_data_db/flow/Helftheuvelweg/')
maaspoort = read_file_to_df('C:/Users/20175848/Documents/waterschap-datachallenge/data/den bosch/sewer_data_db/flow/Maaspoort/')
oud_engelseweg = read_file_to_df('C:/Users/20175848/Documents/waterschap-datachallenge/data/den bosch/sewer_data_db/flow/Oud Engelseweg/')
engelerschans = read_file_to_df('C:/Users/20175848/Documents/waterschap-datachallenge/data/den bosch/sewer_data_db/flow/Engelerschans/')


C:\Users\20175848\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [9]:
name_list = [bokhoven, de_rompert, haarsteeg, helftheuvelweg, maaspoort, oud_engelseweg, engelerschans]

In [5]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [8]:
display_side_by_side(bokhoven.head(), de_rompert.head(), haarsteeg.head(), helftheuvelweg.head()
                     , maaspoort.head(), oud_engelseweg.head(), engelerschans.head())

,Unnamed: 0,datumBeginMeting,datumEindeMeting,datumMonster,historianEenheid,historianKwaliteit,historianTagnummer,hstWaarde
0,0,2018-01-06 23:59:00,2018-01-07 00:00:00,NaN,NaN,100,GBS_DB.RG8180/FI510/Debietmeting.Q,0.0
1,1,2018-01-06 23:58:00,2018-01-06 23:59:00,NaN,NaN,100,GBS_DB.RG8180/FI510/Debietmeting.Q,0.0
2,2,2018-01-06 23:57:00,2018-01-06 23:58:00,NaN,NaN,100,GBS_DB.RG8180/FI510/Debietmeting.Q,0.0
3,3,2018-01-06 23:56:00,2018-01-06 23:57:00,NaN,NaN,100,GBS_DB.RG8180/FI510/Debietmeting.Q,0.0
4,4,2018-01-06 23:55:00,2018-01-06 23:56:00,NaN,NaN,100,GBS_DB.RG8180/FI510/Debietmeting.Q,0.0
,Unnamed: 0,datumBeginMeting,datumEindeMeting,datumMonster,historianEenheid,historianKwaliteit,historianTagnummer,hstWaarde
0,0,2018-01-06 23:59:00,2018-01-07 00:00:00,NaN,m/h,100.0,RWZI_DB/wl/TR/ntk1/1210FIT501/AN.Q,0.0
1,1,2018-01-06 23:58:00,2018-01-06 23:59:00,NaN,m/h,100.0,RWZI_DB/wl/TR/ntk1/1210FIT501/AN.Q,0.0
2,2,2018-01-06 23:57:00,2018-01-06 23:58:00,NaN,m/h,100.0,RWZI_DB/wl/TR/ntk1/1210FIT501/AN.Q,0.0
3,3,2018-01-06 23:56:00,2018-01-06 23:57:00,NaN,m/h,100.0,RWZI_DB/wl/TR/ntk1/1210FIT501/AN.Q,0.0


In [50]:
bokhoven = bokhoven[['datumBeginMeting', 'datumEindeMeting', 'hstWaarde']]
de_rompert = de_rompert[['datumBeginMeting', 'datumEindeMeting', 'hstWaarde']]
haarsteeg = haarsteeg[['datumBeginMeting', 'datumEindeMeting', 'hstWaarde']]
helftheuvelweg = helftheuvelweg[['datumBeginMeting', 'datumEindeMeting', 'hstWaarde']]
maaspoort = maaspoort[['datumBeginMeting', 'datumEindeMeting', 'hstWaarde']]
oud_engelseweg = oud_engelseweg[['datumBeginMeting', 'datumEindeMeting', 'hstWaarde']]
engelerschans = engelerschans[['datumBeginMeting', 'datumEindeMeting', 'hstWaarde']]


engelerschans.head()


,datumBeginMeting,datumEindeMeting,hstWaarde
0,2018-01-06 23:59:00,2018-01-07 00:00:00,2.6
1,2018-01-06 23:58:00,2018-01-06 23:59:00,2.6
2,2018-01-06 23:57:00,2018-01-06 23:58:00,2.6
3,2018-01-06 23:56:00,2018-01-06 23:57:00,2.6
4,2018-01-06 23:55:00,2018-01-06 23:56:00,2.6


In [52]:
bokhoven = bokhoven.rename(columns ={'hstWaarde' : 'bokhoven'})
de_rompert = de_rompert.rename(columns={'hstWaarde' : 'de_rompert'})
haarsteeg = haarsteeg.rename(columns={'hstWaarde' : 'haarsteeg'})
helftheuvelweg = helftheuvelweg.rename(columns={'hstWaarde' : 'helftheuvelweg'})
maaspoort = maaspoort.rename(columns={'hstWaarde' : 'maaspoort'})
oud_engelseweg = oud_engelseweg.rename(columns={'hstWaarde' : 'oud_engelseweg'})
engelerschans = engelerschans.rename(columns={'hstWaarde' : 'engelerschans'})


In [53]:
# display_side_by_side(bokhoven.head(), de_rompert.head(), haarsteeg.head(), helftheuvelweg.head()
#                      , maaspoort.head(), oud_engelseweg.head(), engelerschans.head())

,datumBeginMeting,datumEindeMeting,bokhoven
0,2018-01-06 23:59:00,2018-01-07 00:00:00,0.0
1,2018-01-06 23:58:00,2018-01-06 23:59:00,0.0
2,2018-01-06 23:57:00,2018-01-06 23:58:00,0.0
3,2018-01-06 23:56:00,2018-01-06 23:57:00,0.0
4,2018-01-06 23:55:00,2018-01-06 23:56:00,0.0
,datumBeginMeting,datumEindeMeting,de_rompert
0,2018-01-06 23:59:00,2018-01-07 00:00:00,0.0
1,2018-01-06 23:58:00,2018-01-06 23:59:00,0.0
2,2018-01-06 23:57:00,2018-01-06 23:58:00,0.0
3,2018-01-06 23:56:00,2018-01-06 23:57:00,0.0


In [33]:
for i in name_list:
    print(i.shape)

(863880, 8)
(829320, 8)
(863880, 8)
(863880, 8)
(829320, 8)
(863880, 8)
(855240, 8)


In [64]:
data = pd.merge(left = bokhoven, right = de_rompert,  on=['datumBeginMeting'], how='inner').drop(['datumEindeMeting_x', 'datumEindeMeting_y'], axis = 1)
data = pd.merge(left = data, right = haarsteeg,  on=['datumBeginMeting'], how='inner').drop(['datumEindeMeting'], axis = 1)
data = pd.merge(left = data, right = helftheuvelweg,  on=['datumBeginMeting'], how='inner').drop(['datumEindeMeting'], axis = 1)
data = pd.merge(left = data, right = maaspoort,  on=['datumBeginMeting'], how='inner').drop(['datumEindeMeting'], axis = 1)
data = pd.merge(left = data, right = oud_engelseweg,  on=['datumBeginMeting'], how='inner').drop(['datumEindeMeting'], axis = 1)
data = pd.merge(left = data, right = engelerschans,  on=['datumBeginMeting'], how='inner').drop(['datumEindeMeting'], axis = 1)


In [65]:
data
# name_list = [bokhoven, de_rompert, haarsteeg, helftheuvelweg, maaspoort, oud_engelseweg, engelerschans]

,datumBeginMeting,bokhoven,de_rompert,haarsteeg,helftheuvelweg,maaspoort,oud_engelseweg,engelerschans
0,2018-01-06 23:59:00,0.0,0.0,4.6,1.0,0.0,0.0,2.6
1,2018-01-06 23:58:00,0.0,0.0,4.6,1.0,0.0,0.0,2.6
2,2018-01-06 23:57:00,0.0,0.0,4.6,1.0,0.0,0.0,2.6
3,2018-01-06 23:56:00,0.0,0.0,4.6,1.0,0.0,0.0,2.6
4,2018-01-06 23:55:00,0.0,0.0,4.6,1.0,0.0,0.0,2.6
5,2018-01-06 23:54:00,0.0,0.0,4.6,1.0,0.0,0.0,2.6
6,2018-01-06 23:53:00,0.0,0.0,4.6,1.0,0.0,0.0,2.6
7,2018-01-06 23:52:00,0.0,0.0,4.6,1.0,0.0,0.0,2.6
8,2018-01-06 23:51:00,0.0,0.0,4.6,1.0,0.0,0.0,113.3
9,2018-01-06 23:50:00,0.0,0.0,4.6,1.0,0.0,0.0,136.7
